In [1]:
import keras
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# Setting seed for reproducability
np.random.seed(1234)  
PYTHONHASHSEED = 0
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, recall_score, precision_score
from keras.models import Sequential,model_from_json
from keras.layers import Dense, Dropout, LSTM, Activation
%matplotlib inline
from decimal import Decimal
import warnings
warnings.filterwarnings("ignore")
import pickle
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.preprocessing import Normalizer, MinMaxScaler
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)
from sklearn.metrics import mean_absolute_error
from sklearn import ensemble
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
import xgboost as xgb

Using TensorFlow backend.


In [2]:
df=pd.read_csv('C:\\Users\\lengada1\\NCSU\\ten_skus.csv')
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.weekday_name

day_dummy=pd.get_dummies(df.Day)
df=pd.concat([df,day_dummy],axis=1)
df.drop(['Day','Date'],inplace=True,axis=1)


In [3]:
def split_df(df):
    d={}
    split_by_list=np.unique(df.id.values)
    for sku in split_by_list:
        d[sku]=df[df.id==sku]
    return d

df_dict=split_df(df);

In [4]:
y={}
for sku in list(df_dict.keys()):
    y[sku]=df_dict[sku]['Sales']

In [5]:
X={}
for sku in list(df_dict.keys()):
    X[sku]=df_dict[sku]
    X[sku]=X[sku].drop(['id','DayOfWeek','Customers','High_Var','Luxury','Sales'],axis=1)

In [6]:
for sku in list(df_dict.keys()):
    for obs in range(1,8):
        X[sku]["Sales_T"+str(obs)]=df_dict[sku]['Sales'].shift(obs)

In [7]:
for sku in list(df_dict.keys()):
    X[sku]["Mov_avg"]=pd.rolling_mean(df_dict[sku]['Sales'], window=7).shift(1) 

In [8]:
#Cut lagged vars NAs off top 
cut_lag=7;
for sku in list(df_dict.keys()):
    y[sku]=y[sku][cut_lag:]
    y[sku].reset_index(drop=True, inplace=True)
    
for sku in list(df_dict.keys()):
    X[sku]=X[sku][cut_lag:]
    X[sku].reset_index(drop=True, inplace=True)

In [9]:
X[1].head()

,Open,Promo,SchoolHoliday,Year,Month,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,Sales_T1,Sales_T2,Sales_T3,Sales_T4,Sales_T5,Sales_T6,Sales_T7,Mov_avg
0,1,1,1,2013,1,0,0,0,0,0,1,0,7176.0,0.0,4997.0,4486.0,4327.0,5530.0,0.0,3788.000000
1,1,1,1,2013,1,0,0,0,0,0,0,1,5580.0,7176.0,0.0,4997.0,4486.0,4327.0,5530.0,4585.142857
2,1,1,1,2013,1,0,0,0,0,1,0,0,5471.0,5580.0,7176.0,0.0,4997.0,4486.0,4327.0,4576.714286
3,1,1,1,2013,1,1,0,0,0,0,0,0,4892.0,5471.0,5580.0,7176.0,0.0,4997.0,4486.0,4657.428571
4,1,0,0,2013,1,0,0,1,0,0,0,0,4881.0,4892.0,5471.0,5580.0,7176.0,0.0,4997.0,4713.857143


In [10]:
def RF_model(X_train, X_test, y_train, y_test):
    tree=RandomForestRegressor( random_state=2)
    tree.fit(X_train,y_train)
    pred=tree.predict(X_test)
    pred=pd.DataFrame(pred)
    pred.reset_index(drop=True, inplace=True)
    pred=pred.rename(columns = {0:'RF'})
    return pred

def NN_model(X_train, X_test, y_train, y_test):
    model = Sequential()
    model.add(Dense(20, input_dim=X_train.shape[1], activation='relu'))
    #model.add(Dense(20, activation='relu'))
    model.add(Dense(40, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='relu'))
    model.compile(loss='mse', optimizer='adam', metrics=['mae','mse'])
    model.fit(X_train, y_train, epochs=220, batch_size=25, verbose=0)
    prediction=model.predict(X_test, verbose=0)
    pred=pd.DataFrame(prediction)
    pred.reset_index(drop=True, inplace=True)
    pred=pred.rename(columns = {0:'NN'})
    return pred

def GB_model(X_train, X_test, y_train, y_test):
    params = {'n_estimators': 500, 'max_depth': 40, 'min_samples_split': 20,
          'learning_rate': 0.01, 'loss': 'ls'}
    model = ensemble.GradientBoostingRegressor(**params)
    model.fit(X_train, y_train)
    prediction=model.predict(X_test)
    pred=pd.DataFrame(prediction)
    pred.reset_index(drop=True, inplace=True)
    pred=pred.rename(columns = {0:'GB'})
    return pred

def XG_model(X_train, X_test, y_train, y_test):
    model=xgb.XGBRegressor( #learning_rate =0.1, n_estimators=1000, max_depth=10,
        #min_child_weight=1,gamma=0, subsample=0.8,colsample_bytree=0.8,nthread=4,scale_pos_weight=1,seed=27  
                          )
    model.fit(X_train, y_train)
    prediction=model.predict(X_test)
    pred=pd.DataFrame(prediction)
    pred.reset_index(drop=True, inplace=True)
    pred=pred.rename(columns = {0:'XG'})
    return pred

In [11]:
train_size=800
X_train={}
X_test={}
y_train={}
y_test={}

for sku in list(df_dict.keys()):
    X_train[sku],X_test[sku],y_train[sku],y_test[sku] = X[sku][0:train_size], X[sku][train_size:len(X[sku])], y[sku][0:train_size], y[sku][train_size:len(X[sku])]


In [12]:
std={}
for sku in list(df_dict.keys()):
    std[sku] = preprocessing.StandardScaler().fit(X_train[sku])

for sku in list(df_dict.keys()):
    X_train[sku] = std[sku].transform(X_train[sku])

for sku in list(df_dict.keys()):
    X_test[sku] = std[sku].transform(X_test[sku])

In [13]:
# Fit Models
rf_models={};
for sku in list(df_dict.keys()):
    rf_models[sku]=RF_model( X_train[sku],X_test[sku],y_train[sku],y_test[sku]   )

nn_models={};
for sku in list(df_dict.keys()):
    nn_models[sku]=NN_model( X_train[sku],X_test[sku],y_train[sku],y_test[sku]   )

gb_models={};
for sku in list(df_dict.keys()):
    gb_models[sku]=GB_model( X_train[sku],X_test[sku],y_train[sku],y_test[sku]   )
    
xg_models={};
for sku in list(df_dict.keys()):
    xg_models[sku]=XG_model( X_train[sku],X_test[sku],y_train[sku],y_test[sku]   )

In [14]:
all_pred={}
for sku in list(df_dict.keys()):
    y_test[sku].reset_index(drop=True, inplace=True)
    all_pred[sku]=pd.concat([rf_models[sku],nn_models[sku],gb_models[sku],xg_models[sku],y_test[sku]],axis=1)


In [15]:
def MAE_score(y_true,pred1, pred2,pred3,pred4):
    mae1 = mean_absolute_error(y_true, pred1)
    mae2 = mean_absolute_error(y_true, pred2)
    mae3 = mean_absolute_error(y_true, pred3)
    mae4 = mean_absolute_error(y_true, pred4)
    return [mae1.round(),mae2.round(),mae3.round(),mae4.round()]
MAE_score(y_test[1],rf_models[1],nn_models[1],gb_models[1],xg_models[1])

[322.0, 370.0, 340.0, 348.0]

In [16]:
mae=[]
for sku in list(df_dict.keys()):
    mae.append(MAE_score(all_pred[sku]['Sales'],all_pred[sku]['RF'] ,all_pred[sku]['NN'],all_pred[sku]['GB'],all_pred[sku]['XG'] ) )
mae=pd.DataFrame(mae)
mae=mae.rename(columns = {0:'RF',1:'NN',2:'GB',3:'XG'})

best=mae.idxmin(axis=1)
best.astype('category')
mae["Best"]=best
mae.index = mae.index + 1
mae.loc['mean'] = mae.mean()
mae['range']=mae.max(axis=1)-mae.min(axis=1)
mae

,RF,NN,GB,XG,Best,range
1,322.0,370.0,340.0,348.0,RF,48.0
2,374.0,365.0,380.0,375.0,NN,15.0
3,495.0,621.0,531.0,549.0,RF,126.0
4,714.0,8236.0,709.0,704.0,XG,7532.0
5,301.0,319.0,316.0,294.0,XG,25.0
6,328.0,388.0,333.0,321.0,XG,67.0
7,643.0,736.0,603.0,718.0,GB,133.0
8,518.0,461.0,531.0,560.0,NN,99.0
9,641.0,576.0,613.0,642.0,NN,66.0
10,431.0,430.0,435.0,456.0,NN,26.0


In [17]:
#NOTE, because we are retraining models here, a bad initiatlization could lead to different results. 
X_norm={}
for sku in list(df_dict.keys()):
    X_norm[sku]=np.concatenate((X_train[1],X_test[1]), axis=0)

def best_model(scores,X,y):
    d={}
    for sku in list(df_dict.keys()):
        
        if scores.loc[sku]["Best"]=="RF":
     
            d[sku]=RF_model(X_train[sku],X_norm[sku],y_train[sku],y_test[sku])
                    
        elif scores.loc[sku]["Best"]=="NN":

            d[sku]=NN_model(X_train[sku],X_norm[sku],y_train[sku],y_test[sku])                  
            
        elif scores.loc[sku]["Best"]=="GB":
            d[sku]=GB_model(X_train[sku],X_norm[sku],y_train[sku],y_test[sku])  
            
        elif scores.loc[sku]["Best"]=="XG":
            d[sku]=XG_model(X_train[sku],X_norm[sku],y_train[sku],y_test[sku])
      
    return d     

In [18]:
bm_dict=best_model(mae,X,y)

In [19]:
#MAE of Best Model forecasting over all 935 days.  NN do very poorly here!
mae2=[]
for sku in list(df_dict.keys()):
    mae2.append( mean_absolute_error(y[sku], bm_dict[sku])   )
mae2=pd.DataFrame(mae2)
mae2.index = mae2.index + 1
mae2.round(0)

,0
1,176.0
2,4130.0
3,701.0
4,764.0
5,488.0
6,509.0
7,961.0
8,476.0
9,524.0
10,418.0


In [20]:
sku_pred=pd.concat([ bm_dict[1],bm_dict[2],bm_dict[3],bm_dict[4],bm_dict[5],bm_dict[6],bm_dict[7],bm_dict[8],bm_dict[9],bm_dict[10],
],axis=1) 
sku_pred['Total'] = sku_pred.sum(axis=1)
sku_pred.columns = ['1','2','3','4','5','6','7','8','9','10','Total']
sku_pred.to_csv('C:\\Users\\lengada1\\NCSU\\prediction_skus.csv')
sku_pred.shape

(935, 11)